In [ ]:
import os
import pickle
import pandas as pd

def get_model(model_name: str = "deploy/model.pkl") -> any:
    # Get the current directory of the script
    current_directory = ""
    # Construct the relative path to the model file
    model_path = os.path.join(current_directory, model_name)
    # Check if the model file exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file '{model_name}' not found in the current directory.")
    
    # Load the model
    with open (model_path, 'rb') as m:
        model = pickle.load(m)
    print(type(model))
    return model

In [ ]:
MODEL = get_model()

In [ ]:
with open('deploy/input_data.txt', 'r') as file:
    input_data = eval(file.read())



In [ ]:
new_data = pd.DataFrame.from_dict(input_data, orient="index").T
if 'Datum' in new_data.columns:
    # Convert Datum to datetime and other columns to numeric
    new_data["Datum"] = pd.to_datetime(new_data["Datum"], dayfirst=True)
    columns_to_convert = new_data.columns.difference(["Datum"])
else:
    columns_to_convert = new_data.columns

def convert_to_numeric(column):
    return pd.to_numeric(column, errors="coerce")

new_data[columns_to_convert] = new_data[columns_to_convert].apply(
    convert_to_numeric
)
print(new_data)
# Make predictions
predictions = MODEL.predict_proba(new_data) 

# Convert predictions to a list
predictions_list = float(predictions[:, 1][0])

    

In [ ]:
predictions_list

In [ ]:
predictions

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from hydra.utils import instantiate
from hydra import compose, initialize

from my_logger.custom_logger import  logger
from ProcessData.data_splitter import data_splitter
from ProcessData.dataset import Dataset

initialize(config_path="conf/", version_base=None)
cfg = compose(config_name="config.yaml")


data_pipeline = instantiate(cfg.data_pipeline)
df = pd.read_csv(cfg.Data_Source)
df = data_pipeline.apply(df.copy())
dataset = Dataset(data=df, data_splitter=data_splitter)

In [ ]:
row = dataset.X.iloc[-100]

In [ ]:
pd.DataFrame(row).T

In [ ]:
MODEL.predict_proba(pd.DataFrame(row).T)

In [ ]:
pd.DataFrame(new_data)